###Target:
Basic architecture with good potential 

###Results:
1. Parameters: 15,918
2. Best Train Accuracy: 99.24%
3. Best Test Accuracy: 98.86%

###Analysis:
1. model seems to show promise with Highest Train and Test accuracy not differing much.
2. Slight Over-fitting in my opinion as Train accuracy is increasing with epochs but Test accuracy got stagnent after 6-7 epochs
3. Need to decrease the number of parameters as well.
 

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        #block1
        self.conv1 = nn.Conv2d(1, 10, 3) #26
        #self.Batchnorm1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 18, 3) #24
        #self.Batchnorm2 = nn.BatchNorm2d(18)
        self.conv3 = nn.Conv2d(18,18,3) #22
        #self.Batchnorm3 = nn.BatchNorm2d(18)
        #self.dp3 = nn.Dropout(p = 0.20)

        #transition block
        self.pool1 = nn.MaxPool2d(2, 2) #11
        self.pool1trns = nn.Conv2d(18, 10, 1)#11
        #self.Batchnormtrns1 = nn.BatchNorm2d(10)
        
        #block2
        self.conv4 = nn.Conv2d(10, 18, 3)#9
        #self.Batchnorm4 = nn.BatchNorm2d(18)
        #self.dp4 = nn.Dropout(p = 0.20)
        self.conv5 = nn.Conv2d(18, 26, 3)#7
        #self.Batchnorm5 = nn.BatchNorm2d(26)
        #self.dp5 = nn.Dropout(p = 0.20)

        #block3
        self.pool2trns = nn.Conv2d(26,10,1) #7
        #self.Batchnormtrns2 = nn.BatchNorm2d(10)
        self.conv6 = nn.Conv2d(10,10,7) #1


    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        #x = self.Batchnorm1(x)
        
        #x = self.dropout(x)
        x = self.conv2(x)
        x = F.relu(x)
        #x = self.Batchnorm2(x)

        
        #x = self.dropout(x)
        x = self.conv3(x)
        x = F.relu(x)
        #x = self.Batchnorm3(x)
        #x = self.dp3(x)
        
        #x = self.dropout(x)
        x = self.pool1(x)
        x = self.pool1trns(x)
        x = F.relu(x)
        #x = self.Batchnormtrns1(x)
        
        x = self.conv4(x)
        x = F.relu(x)
        #x = self.Batchnorm4(x)
        #x = self.dp4(x)


        x = self.conv5(x)
        x = F.relu(x)
       #x = self.Batchnorm5(x)
       #x = self.dp5(x)
        
        x = self.pool2trns(x)
        x = F.relu(x)
       #x = self.Batchnormtrns2(x)


        x = self.conv6(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
            Conv2d-2           [-1, 18, 24, 24]           1,638
            Conv2d-3           [-1, 18, 22, 22]           2,934
         MaxPool2d-4           [-1, 18, 11, 11]               0
            Conv2d-5           [-1, 10, 11, 11]             190
            Conv2d-6             [-1, 18, 9, 9]           1,638
            Conv2d-7             [-1, 26, 7, 7]           4,238
            Conv2d-8             [-1, 10, 7, 7]             270
            Conv2d-9             [-1, 10, 1, 1]           4,910
Total params: 15,918
Trainable params: 15,918
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.25
Params size (MB): 0.06
Estimated Total Size (MB): 0.31
---------------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    #pbar = tqdm(train_loader)
    train_loss = 0 
    train_correct = 0
    #scheduler.step()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        train_correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss /= len(test_loader.dataset)
    print('Epoch: {:.0f},LR: {}.\nTrain set: train Average loss: {:.4f}, train_Accuracy: {}/{} ({:.4f}%)\n'.format(
        epoch,optimizer.param_groups[0]['lr'],train_loss, train_correct, len(train_loader.dataset),
        100. * train_correct / len(train_loader.dataset)))
        

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred_test = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred_test.eq(target.view_as(pred_test)).sum().item()

    test_loss /= len(test_loader.dataset)

    
    print('Test set: test Average loss: {:.4f}, test Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1,LR: 0.01.
Train set: train Average loss: 8.7486, train_Accuracy: 27526/60000 (45.8767%)

Test set: test Average loss: 0.2197, test Accuracy: 9329/10000 (93.2900%)

Epoch: 2,LR: 0.01.
Train set: train Average loss: 0.9127, train_Accuracy: 57221/60000 (95.3683%)

Test set: test Average loss: 0.1030, test Accuracy: 9675/10000 (96.7500%)

Epoch: 3,LR: 0.01.
Train set: train Average loss: 0.5564, train_Accuracy: 58328/60000 (97.2133%)

Test set: test Average loss: 0.0997, test Accuracy: 9685/10000 (96.8500%)

Epoch: 4,LR: 0.01.
Train set: train Average loss: 0.4447, train_Accuracy: 58665/60000 (97.7750%)

Test set: test Average loss: 0.0570, test Accuracy: 9818/10000 (98.1800%)

Epoch: 5,LR: 0.01.
Train set: train Average loss: 0.3705, train_Accuracy: 58862/60000 (98.1033%)

Test set: test Average loss: 0.0511, test Accuracy: 9832/10000 (98.3200%)

Epoch: 6,LR: 0.01.
Train set: train Average loss: 0.3174, train_Accuracy: 58997/60000 (98.3283%)

Test set: test Average loss: 0.0461, 